In [3]:
import tensorflow as tf
from sklearn.metrics import confusion_matrix
import numpy as np
import pandas as pd
from pandas import DataFrame
import matplotlib.pyplot as plt
import seaborn as sns
import skimage as ski
from skimage.feature import hog
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import json

import time
from datetime import timedelta
import math

import h5py

%matplotlib inline

/Users/Aidan/anaconda/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## Load Data

In [4]:
d_train = h5py.File('./train_m_28/train_2.h5', 'r')
d_test = h5py.File('./train_m_28/test_2.h5', 'r')
d_val = h5py.File('./train_m_28/validation_1.h5', 'r')

In [85]:
Xtrain = d_train['train']['images']
ytrain = d_train['train']['labels']

Xtest = d_test['test']['images']

Xval = d_val['validation']['images']
yval = d_val['validation']['labels']

print(\
'Training samples:\t{}\n\
Test samples:\t\t{}\n\
Validation Samples:\t{}'.format(Xtrain.shape[0], Xtest.shape[0], Xval.shape[0]))


Training samples:	192171
Test samples:		12479
Validation Samples:	6309
6309


## Preprocess

In [6]:
Xtrain

<HDF5 dataset "images": shape (192171, 784), type "<f8">

In [9]:
Xtrain_new = []
Xval_new = []
Xtest_new = []

for idx, x in enumerate(Xtrain):
    x = x.reshape(28,28)
    xg = hog(x, orientations=8, pixels_per_cell=(4, 4), 
             cells_per_block=(1, 1), feature_vector=True)
    Xtrain_new.append(xg)
    if idx % 10000 == 0:
        print(idx)
Xtrain_new = np.array(Xtrain_new).astype('float32')
print(Xtrain_new.data.shape)


for idx, x in enumerate(Xval):
    x = x.reshape(28,28)
    xg = hog(x, orientations=8, pixels_per_cell=(4, 4),
                    cells_per_block=(1, 1), feature_vector=True)
    Xval_new.append(xg)
    if idx % 1000 == 0:
        print(idx)
Xval_new = np.array(Xval_new).astype('float32')
print(Xval_new.data.shape)

for idx, x in enumerate(Xtest):
    x = x.reshape(28,28)
    xg = hog(x, orientations=8, pixels_per_cell=(4, 4),
                    cells_per_block=(1, 1), feature_vector=True)
    Xtest_new.append(xg)
    if idx % 1000 == 0:
        print(idx)
Xtest_new = np.array(Xtest_new).astype('float32')
print(Xtest_new.data.shape)

/Users/Aidan/anaconda/lib/python3.6/site-packages/skimage/feature/_hog.py:119: skimage_deprecation: Default value of `block_norm`==`L1` is deprecated and will be changed to `L2-Hys` in v0.15
  'be changed to `L2-Hys` in v0.15', skimage_deprecation)


0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
(192171, 392)
0
1000
2000
3000
4000
5000
6000
(6309, 392)
0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
(12479, 392)


In [10]:
algorithm = 'kd_tree'
n_neighbors = 10

print('Fitting model....')
model = KNeighborsClassifier(n_neighbors=n_neighbors, algorithm=algorithm)
model.fit(Xtrain_new, ytrain)


Fitting model....


KNeighborsClassifier(algorithm='kd_tree', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=10, p=2,
           weights='uniform')

In [11]:
# compute accuracy
print('Computing acc on validation set...')
preds = model.predict(Xval_new)
acc = accuracy_score(yval, preds)
print("Final accuracy: {}".format(acc))

Computing acc on validation set...
Final accuracy: 0.2545569820890791


In [12]:
#compute test set labels 
predicted_test_labels = model.predict(Xtest_new)

In [81]:
#convert to ints for submission
predicted_test_labels_ints = [int(x) for x in predicted_test_labels]

In [82]:
Xs = range(1,12801)
Ys = predicted_test_labels_ints
#make Ys longer
while len(Ys) < 12801:
    Ys.append(1)
print(len(Ys))
XYs = [[x,y] for x,y in zip(Xs,Ys)]
#XYs.insert(0, ['id','predicted'])
#numpy.savetxt("/Users/Aidan/Desktop/submission1.csv", a, delimiter=",")

12801


In [83]:
np.savetxt("/Users/Aidan/Desktop/submission801.csv", XYs, delimiter=",", 
            fmt=('%d, %d'), header='id,predicted')
#print(Ys[0])

## Random vis

In [84]:
test = json.load(open('./data/test.json'))
i = 1
print('Picture # : ' + str(i))
p = Xtrain[i]
p_plot = plt.imshow(p)
#print(len(Xtrain))
#print(len(ytrain))
print("Label is : " + str(ytrain[i]))
print(training['images'][i])
print(training['annotations'][i])

FileNotFoundError: [Errno 2] No such file or directory: './data/test.json'